In [1]:
import triton

In [3]:
import triton.language as tl

@triton.jit
def print_grid():
    pid = tl.program_id(0)
    tl.device_print("pid", pid)

In [4]:
def grid(meta):
    return (4,)

print_grid[grid]()

pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx () pid: 2
pid (2, 0, 0) idx ()

In [10]:
import torch

@triton.jit
def add_kernel(x_ptr, y_ptr, z_ptr, size, block_size : tl.constexpr):
    pid = tl.program_id(0)
    offsets = tl.arange(0, block_size) + pid * block_size
    mask = offsets < size
    
    x = tl.load(x_ptr + offsets, mask)
    y = tl.load(y_ptr+ offsets, mask)
    z = x + y
    tl.store(z_ptr + offsets, z, mask)
    
def add(x, y):
    z = torch.empty_like(x, device="cuda")
    size = z.numel()
    
    def grid(meta):
        return (triton.cdiv(size, meta["block_size"]), )
    
    add_kernel[grid](x, y, z, size, 1024)
    return z
    
size = 2**16
x = torch.randn(size, device="cuda")
y = torch.randn(size, device="cuda")
a = add(x, y)
b = x + y
assert torch.allclose(a, b)

In [ ]:
import torch
import triton 
import triton.language

@triton.jit
def combine_add(a, b):
    return